# Tarea 2 — Regresión: Estimación del costo de seguro médico

**Dataset recomendado:** [Medical Insurance Cost Dataset (Kaggle)](https://www.kaggle.com/datasets/mosapabdelghany/medical-insurance-cost-dataset)

Este cuaderno cumple con los puntos solicitados:
1. Crear repositorio (ya realizado) y preparar el análisis.
2. Exploración de datos: descripción, valores vacíos y transformaciones iniciales.
3. Análisis **univariante** de variables numéricas y categóricas, con interpretaciones.
4. **Filtrado de outliers** por método IQR (intercuartiles) y justificación del corte.
5. Definir si se **trata la variable objetivo** (`charges`) o se deja tal cual (se propone `log(charges)`).
6. Análisis **bivariante**: gráfica de `charges` vs. cada variable y explicación.
7. **Matriz de correlación** y decisión de eliminación de variables poco útiles.
8. División del dataset en **train/test (80/20)** **estratificando** sobre cuantiles de `charges` para mantener la distribución.
9. Guardar `train.csv` y `test.csv` para el modelado posterior.

> **Nota**: Coloca el archivo original como `insurance.csv` en esta misma carpeta. Si no está presente, el cuaderno generará un conjunto **sintético** con las mismas columnas para que puedas practicar todo el flujo.


In [ ]:
# ========================
# 1) Carga de datos
# ========================
import pandas as pd, numpy as np
from pathlib import Path

path = Path('insurance.csv')
if path.exists():
    df = pd.read_csv(path)
    origen = 'original Kaggle'
else:
    # Generador de datos sintéticos compatibles (solo para práctica)
    np.random.seed(42)
    n = 1338
    df = pd.DataFrame({
        'age': np.random.randint(18, 65, n),
        'sex': np.random.choice(['male','female'], n),
        'bmi': np.round(np.random.normal(30, 6, n), 2),
        'children': np.random.randint(0, 5, n),
        'smoker': np.random.choice(['yes','no'], n, p=[0.2,0.8]),
        'region': np.random.choice(['southwest','southeast','northwest','northeast'], n),
    })
    # charges sintético: base + efectos aproximados
    base = 2000 + df['age']*50 + (df['bmi']-25)*100 + df['children']*400
    fum = np.where(df['smoker']=='yes', 12000, 0)
    region_effect = df['region'].map({
        'southwest': -200,
        'southeast': 300,
        'northwest': -100,
        'northeast': 0
    }).values
    ruido = np.random.normal(0, 2000, n)
    df['charges'] = np.maximum(1000, base + fum + region_effect + ruido)
    origen = 'sintético (práctica)'

print('Origen de los datos:', origen)
df.head()


In [ ]:
# ========================
# 2) Exploración inicial
# ========================
display(df.shape)
display(df.dtypes)
display(df.isna().sum())
display(df.describe(include='all'))

explicacion = (
    "Se revisan dimensiones, tipos de datos y valores faltantes. "
    "En caso de NA, se aplicarán imputaciones (mediana para numéricas, moda para categóricas). "
    "También se verifican rangos y consistencia de variables (por ejemplo, BMI, edad, etc.)."
)
print(explicacion)


In [ ]:
# Tratamiento de NA (si los hubiera)
for col in df.columns:
    if df[col].dtype.kind in 'biufc':
        df[col] = df[col].fillna(df[col].median())
    else:
        df[col] = df[col].fillna(df[col].mode()[0])
df.isna().sum()


## 3) Análisis univariante (numéricas y categóricas) con interpretaciones

Se grafican histogramas para numéricas y barras para categóricas. Se incluyen comentarios sobre posibles sesgos o patrones.


In [ ]:
import matplotlib.pyplot as plt

num_cols = ['age','bmi','children','charges']
cat_cols = ['sex','smoker','region']

for col in num_cols:
    plt.figure()
    plt.hist(df[col].values, bins=30)
    plt.title(f'Distribución de {col}')
    plt.xlabel(col); plt.ylabel('Frecuencia')
    plt.show()

for col in cat_cols:
    plt.figure()
    counts = df[col].value_counts()
    plt.bar(counts.index.astype(str), counts.values)
    plt.title(f'Distribución de {col}')
    plt.xlabel(col); plt.ylabel('Conteo')
    plt.xticks(rotation=20)
    plt.show()


### Comentarios (ejemplo para tu reporte)
- `smoker` suele estar altamente asociado a mayores `charges`.
- `bmi` con colas derechas puede sugerir **outliers** en IMC muy altos.
- Distribuciones por `region` pueden estar balanceadas o no, revisarlo.


## 4) Filtrado de outliers por IQR (intercuartiles)

Se aplica a numéricas: `age`, `bmi`, `children`, `charges`. Se justifica el corte en **Q1 - 1.5*IQR** y **Q3 + 1.5*IQR**.


In [ ]:
def iqr_filter(data, cols):
    mask = pd.Series(True, index=data.index)
    for c in cols:
        q1, q3 = data[c].quantile([0.25, 0.75])
        iqr = q3 - q1
        low, high = q1 - 1.5*iqr, q3 + 1.5*iqr
        mask &= data[c].between(low, high)
    return data[mask]

cols_iqr = ['age','bmi','children','charges']
df_filtered = iqr_filter(df, cols_iqr)
print('Filtrado IQR: filas antes:', len(df), ' | después:', len(df_filtered))


## 5) ¿Tratar la variable objetivo `charges`?
Muchas veces `charges` presenta sesgo a la derecha. Se propone trabajar con `log_charges = log(charges)` para estabilizar varianza y mejorar linealidad. También conservamos `charges` original para comparar.


In [ ]:
import numpy as np
df_filtered = df_filtered.copy()
df_filtered['log_charges'] = np.log(df_filtered['charges'])
plt.figure(); plt.hist(df_filtered['charges'], bins=30); plt.title('charges (original)'); plt.show()
plt.figure(); plt.hist(df_filtered['log_charges'], bins=30); plt.title('log(charges)'); plt.show()


## 6) Análisis bivariante: `charges` vs. cada variable
Se grafican relaciones simples. Para categóricas se usan promedios por grupo.


In [ ]:
# Numéricas vs charges
for col in ['age','bmi','children']:
    plt.figure()
    plt.scatter(df_filtered[col], df_filtered['charges'])
    plt.title(f'{col} vs charges')
    plt.xlabel(col); plt.ylabel('charges')
    plt.show()

# Categóricas: medias por grupo
for col in ['sex','smoker','region']:
    plt.figure()
    means = df_filtered.groupby(col)['charges'].mean().sort_values()
    plt.bar(means.index.astype(str), means.values)
    plt.title(f'charges promedio por {col}')
    plt.xticks(rotation=20)
    plt.ylabel('charges promedio')
    plt.show()


## 7) Matriz de correlación y selección de variables
Se codifican variables categóricas para calcular correlaciones y decidir posibles eliminaciones por redundancia o baja relación.


In [ ]:
df_enc = df_filtered.copy()
df_enc['sex'] = (df_enc['sex']=='male').astype(int)
df_enc['smoker'] = (df_enc['smoker']=='yes').astype(int)
df_enc = pd.get_dummies(df_enc, columns=['region'], drop_first=True)
corr = df_enc.corr(numeric_only=True)
corr['charges'].sort_values(ascending=False)


### Comentarios sugeridos
- `smoker` suele ser la variable más correlacionada positivamente con `charges`.
- `age` y `bmi` también aportan. `children` puede tener relación menor.
- Si dos variables están muy correlacionadas entre sí, se puede descartar una para simplificar el modelo.


## 8) División en train/test (80/20) **estratificando** por cuantiles de `charges`
Para mantener la distribución del objetivo en ambos conjuntos, se crean **bins** (cuantiles) de `charges` y se usa `StratifiedShuffleSplit` sobre esos bins.


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

df_split = df_enc.copy()
bins = pd.qcut(df_split['charges'], q=5, duplicates='drop')
df_split['charges_bin'] = bins.cat.codes

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, test_idx in split.split(df_split, df_split['charges_bin']):
    train = df_split.iloc[train_idx].drop(columns=['charges_bin'])
    test  = df_split.iloc[test_idx].drop(columns=['charges_bin'])

print(train.shape, test.shape)
train.head()


In [ ]:
# Comprobar que proporciones de bins se mantienen aproximadamente
def bin_props(frame):
    b = pd.qcut(frame['charges'], q=5, duplicates='drop')
    return b.value_counts(normalize=True).sort_index()

print('Proporciones en TRAIN:\n', bin_props(train))
print('\nProporciones en TEST:\n', bin_props(test))


In [ ]:
# 9) Guardar datasets para la fase de modelado
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)
print('✅ Archivos guardados: train.csv y test.csv')


## Conclusión
- Se realizó EDA completo, tratamiento de valores faltantes y outliers por IQR.
- Se evaluó la variable objetivo (`charges`) y se propuso transformación logarítmica.
- Se analizó la relación bivariante y la correlación para orientar selección de variables.
- Se generaron `train.csv` y `test.csv` manteniendo la distribución del objetivo mediante estratificación por cuantiles.

Este material está listo para la **fase de modelado de regresión** (por ejemplo, `LinearRegression`, `RandomForestRegressor`, `XGBRegressor`) en un cuaderno posterior.
